In [1]:
import numpy as np
import pandas as pd

In [2]:
from tqdm import tqdm

In [3]:
import pymorphy2
import re

In [4]:
data = pd.read_csv("/Users/alexajax/Downloads/test_1.csv")

In [5]:
dlabel = data[data.person_id.values!=-1]
dunlabel = data[data.person_id.values==-1]

In [8]:
import nltk
from nltk.corpus import stopwords as sw 
#nltk.download('stopwords', halt_on_error=False)

stopwords = sw.words("russian")

In [26]:
data[data.person_id.values!=-1]

,note_id,text,person_id
6,203495,Кончаю II т. Сошного письма с утомленным и под...,629
8,103374,На днях прирезала одного цыпленка (ушиб крылыш...,384
11,75608,... 4-ю четверть окончил «круглым» [отличником...,324
13,94397,Вчера я была у Катюши Горянской. Сначала судил...,358
14,99005,Пятница. Теперь командует Черноморским флотом ...,373
18,166870,Заходила М.В.Кістяківська — дякувала за пораду...,402
19,182922,"Однaжды Мaринa<com id=""14805283844198""/> скaзa...",511
20,222660,В воскресенье. Приехала Ира.,566
21,168604,"<...> Мы все с Таруханом и Нару, кроме Порумы ...",454
23,117103,Комаровка. \nДорогой Олег! \nСпасибо теб...,439


In [27]:
data[data.person_id.values!=-1].iloc[0].text

'Кончаю II т. Сошного письма с утомленным и подавленным духом. Кому это нужно теперь? Скоро ли понадобится, да и вообще нужны в России такие работы? Нужно побольше диссертаций, не для науки, а для ученых степеней, которых не хватает многим доцентам и даже профессорам.     \nВ воскресенье у меня был Л. М. Сухотин<com id="1481900746415"/>. Его присутствие и спокойная беседа с ним успокаивают меня и до некоторой степени возвращают мне самообладание. По своей выдержке и воспитанности всех сторон души он очень мало похож на русского. Во всяком случае в России таких людей очень мало.'

## Preprocessing

In [9]:
morph = pymorphy2.MorphAnalyzer()

stem_sentence = lambda x: list(map(lambda x: morph.parse(x)[0].normal_form, x))
lower_sentence = lambda x: list(map(lambda x: x.lower(), x))

In [10]:
dlabel.text = list(map(lambda x: re.sub("[\?]", " ? ", x), dlabel.text))
print("OK!")
dlabel.text = list(map(lambda x: re.sub("[\!]", " ! ", x), dlabel.text))
print("OK!")
dlabel.text = list(map(lambda x: re.sub("[^А-Яа-я\!\?]", " ", x), dlabel.text))
dlabel.text = list(map(lambda x: re.sub("[a]", "а", x), dlabel.text))
dlabel.text = list(map(lambda x: re.sub("[^А-Яа-я]", " ", x), dlabel.text))
print("OK!")
dlabel.text = list(map(lambda x: re.sub("[ё]", "е", x), dlabel.text))
print("OK!")
dlabel.text = list(map(lambda x: x.split(), dlabel.text))
print("OK!")
dlabel.text = list(map(lower_sentence, dlabel.text))
print("OK!")
dlabel.text = list(map(stem_sentence, dlabel.text))
print("OK!")

/Users/alexajax/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


OK!
OK!
OK!
OK!
OK!
OK!
OK!


In [11]:
dunlabel.text = list(map(lambda x: re.sub("[\?]", " ? ", x), dunlabel.text))
print("OK!")
dunlabel.text = list(map(lambda x: re.sub("[\!]", " ! ", x), dunlabel.text))
print("OK!")
dunlabel.text = list(map(lambda x: re.sub("[^А-Яа-я\!\?]", " ", x), dunlabel.text))
dunlabel.text = list(map(lambda x: re.sub("[a]", "а", x), dunlabel.text))
dunlabel.text = list(map(lambda x: re.sub("[^А-Яа-я]", " ", x), dunlabel.text))
print("OK!")
dunlabel.text = list(map(lambda x: re.sub("[ё]", "е", x), dunlabel.text))
print("OK!")
dunlabel.text = list(map(lambda x: x.split(), dunlabel.text))
print("OK!")
dunlabel.text = list(map(lower_sentence, dunlabel.text))
print("OK!")
dunlabel.text = list(map(stem_sentence, dunlabel.text))
print("OK!")

/Users/alexajax/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


OK!
OK!
OK!
OK!
OK!
OK!
OK!


## Build vocab

In [12]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [13]:
vocab = set(flatten(list(map(lambda x: x.split(), list(map(lambda x: " ".join(x), dlabel.text)) + list(map(lambda x: " ".join(x), dunlabel.text))))))

## Vectorizer

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
lmodel = TfidfVectorizer(vocabulary=vocab, stop_words=stopwords)
umodel = TfidfVectorizer(vocabulary=vocab, stop_words=stopwords)

In [16]:
dlabel.text = list(map(lambda x: " ".join(x), dlabel.text))
dunlabel.text = list(map(lambda x: " ".join(x), dunlabel.text))

/Users/alexajax/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [17]:
lmatrix = lmodel.fit_transform(dlabel.text.values)
umatrix = umodel.fit_transform(dunlabel.text.values)

## RF

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
from sklearn.ensemble import RandomForestClassifier

building tree 1 of 150
building tree 2 of 150
building tree 3 of 150
building tree 4 of 150
building tree 5 of 150
building tree 6 of 150
building tree 7 of 150
building tree 8 of 150
building tree 9 of 150
building tree 10 of 150
building tree 11 of 150
building tree 12 of 150
building tree 13 of 150
building tree 14 of 150
building tree 15 of 150
building tree 16 of 150
building tree 17 of 150
building tree 18 of 150
building tree 19 of 150
building tree 20 of 150
building tree 21 of 150
building tree 22 of 150
building tree 23 of 150
building tree 24 of 150


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   33.0s


building tree 25 of 150
building tree 26 of 150
building tree 27 of 150
building tree 28 of 150
building tree 29 of 150
building tree 30 of 150
building tree 31 of 150
building tree 32 of 150
building tree 33 of 150
building tree 34 of 150
building tree 35 of 150
building tree 36 of 150
building tree 37 of 150
building tree 38 of 150
building tree 39 of 150
building tree 40 of 150
building tree 41 of 150
building tree 42 of 150
building tree 43 of 150
building tree 44 of 150
building tree 45 of 150
building tree 46 of 150
building tree 47 of 150
building tree 48 of 150
building tree 49 of 150
building tree 50 of 150
building tree 51 of 150
building tree 52 of 150
building tree 53 of 150
building tree 54 of 150
building tree 55 of 150
building tree 56 of 150
building tree 57 of 150
building tree 58 of 150
building tree 59 of 150
building tree 60 of 150
building tree 61 of 150
building tree 62 of 150
building tree 63 of 150
building tree 64 of 150
building tree 65 of 150
building tree 66

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.9min


building tree 121 of 150
building tree 122 of 150
building tree 123 of 150
building tree 124 of 150
building tree 125 of 150
building tree 126 of 150
building tree 127 of 150
building tree 128 of 150
building tree 129 of 150
building tree 130 of 150
building tree 131 of 150
building tree 132 of 150
building tree 133 of 150
building tree 134 of 150
building tree 135 of 150
building tree 136 of 150
building tree 137 of 150
building tree 138 of 150
building tree 139 of 150
building tree 140 of 150
building tree 141 of 150
building tree 142 of 150
building tree 143 of 150
building tree 144 of 150
building tree 145 of 150
building tree 146 of 150
building tree 147 of 150
building tree 148 of 150
building tree 149 of 150
building tree 150 of 150


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=-1, oob_score=False,
            random_state=None, verbose=3, warm_start=False)

In [19]:
answer = rf.predict(umatrix)

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   26.4s
[Parallel(n_jobs=8)]: Done 150 out of 150 | elapsed:   35.6s finished


In [ ]:
from sklearn.linear import LogisticRegression

In [20]:
dunlabel["person_id"] = answer

/Users/alexajax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
dunlabel.to_csv("subm2.csv", index=None)

In [23]:
np.save("lmatrix.npy", lmatrix)

In [26]:
np.save("umatrix.npy", umatrix)

In [55]:
np.save("y.npy", dlabel.person_id.values)

## XGBoost

In [50]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/Users/alexajax/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/alexajax/anaconda3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [15]:
ans = np.load("../answers.npy")

array([[304, 501, 999, ..., 448, 302, 318],
       [304, 304, 999, ..., 448, 319, 318],
       [304, 304, 999, ..., 448, 450, 318],
       ..., 
       [304, 304, 999, ..., 342, 801, 318],
       [304, 304, 999, ..., 448, 302, 318],
       [304, 501, 999, ..., 374, 450, 318]])

In [19]:
for i in range(len(ans)):
    dunlabel["person_id"] = ans[i]
    pd.concat([dlabel, dunlabel]).to_csv("subms/subm{}.csv".format(i), index=None)

/Users/alexajax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


/Users/alexajax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
pd.concat([dlabel, dunlabel]).to_csv("subm2.csv",index=None)

## Serious stuff begins here...

In [148]:
from sklearn.naive_bayes import GaussianNB

In [159]:
from sklearn.preprocessing import OneHotEncoder

In [173]:
encoder = OneHotEncoder()
encoder.fit_transform(dlabel.person_id).shape

/Users/alexajax/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(1, 40875)